# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: fetch latest code</h2>
            <span style="color:#f71;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in a Cursor Terminal, run:<br/>
            <code>uv sync</code><br/>
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use high end models GPT 5, Claude 4.5 Sonnet, Gemini 2.5 Pro, Grok 4, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please pick lower cost models like gpt-5-nano.
            </span>
        </td>
    </tr>
</table>

In [14]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display

In [15]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)
Google API Key exists and begins AI
Grok API Key not set (and this is optional)


In [16]:
# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)

In [17]:
# OPENAI_MODEL = "gpt-5"
# CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
# GROK_MODEL = "grok-4"
# GEMINI_MODEL = "gemini-2.5-pro"

# Want to keep costs ultra-low? Uncomment these lines:

OPENAI_MODEL = "gpt-5-nano"
CLAUDE_MODEL = "claude-3-5-haiku-latest"
GROK_MODEL = "grok-4-fast-non-reasoning"
GEMINI_MODEL = "gemini-2.5-flash-lite"

## PLEASE NOTE:

We will be writing a solution to convert Python into efficient, optimized C++ code for your machine, which can be compiled to native machine code and executed.

It is not necessary for you to execute the code yourself - that's not the point of the exercise!

But if you would like to (because it's satisfying!) then I'm including the steps here. Very optional!

As an alternative, I'll also show you a website where you can run the C++ code.

In [18]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Windows',
  'arch': 'AMD64',
  'release': '10',
  'version': '10.0.19045',
  'kernel': '10',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'x86_64-w64-mingw32'},
 'package_managers': ['winget'],
 'cpu': {'brand': 'Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz',
  'cores_logical': 8,
  'cores_physical': 4,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'gcc.EXE (Rev2, Built by MSYS2 project) 14.2.0',
   'g++': 'g++.EXE (Rev2, Built by MSYS2 project) 14.2.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': ''},
  'linkers': {'ld_lld': ''}}}

In [19]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))
    

Short answer:
- You don’t need to install a compiler. Your system already has GCC (g++.EXE) from MSYS2, which is a working C++ compiler on Windows.

What to do right now (simplest path)

Option A: Compile and run manually (works best in an MSYS2/MINGW64 shell or any Windows terminal where g++ is in PATH)
- Open a terminal that has g++ in PATH (e.g., MSYS2 MinGW 64-bit shell, or a Windows Command Prompt/PowerShell if MSYS2’s bin is in PATH).
- Make sure main.cpp is in your current directory.
- Compile with optimized settings (fastest runtime):
  - g++ -O3 -std=c++17 main.cpp -o main.exe
  - If you want to push performance further and your CPU supports it, you can add -march=native -mtune=native:
    - g++ -O3 -march=native -mtune=native -std=c++17 main.cpp -o main.exe
- Run:
  - In MSYS2 shell: ./main.exe
  - In Windows CMD/PowerShell: main.exe

Option B: Python snippet (specifically for you, with fastest practical optimization)
- Use these exact command lists:

compile_command = ["g++", "-O3", "-std=c++17", "main.cpp", "-o", "main.exe"]
# If you want the extra aggressive tuning (may not matter for short runs or some CPUs):
# compile_command = ["g++", "-O3", "-march=native", "-mtune=native", "-std=c++17", "main.cpp", "-o", "main.exe"]

run_command = ["main.exe"]

- Then in Python:
  - compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
  - run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
  - output = run_result.stdout

Notes and quick checks
- Your system info shows: gcc.EXE and g++.EXE (MSYS2 project) 14.2.0 are installed. That means you already have a working GCC toolchain.
- If g++ isn’t found in your PATH in a plain Windows terminal, run from the MSYS2 MinGW 64-bit shell or add the appropriate MSYS2 bin directory to your PATH (typically something like C:\msys64\mingw64\bin).
- If your code uses newer C++ features, -std=c++17 is included above; you can switch to -std=c++20 if needed.

If you ever want to install a compiler from scratch (only if you don’t have one)
- In case g++ is not available, the simplest route on Windows is:
  - Install MSYS2 (msys2.org)
  - Open the MSYS2 MinGW 64-bit console and run:
    - pacman -Syu
    - pacman -S --needed base-devel mingw-w64-x86_64-toolchain
  - Ensure C:\msys64\mingw64\bin is on your PATH (or use the MSYS2 shell to compile).
  - Then use the same compile_command and run_command as above.

If you’d like, tell me whether you’re running in a pure Windows CMD/PowerShell environment or inside the MSYS2 shell, and I’ll tailor the exact run_command you should use.

## If you need to install something

If you would like to, please follow GPTs instructions! Then rerun the analysis afterwards (you might need to Restart the notebook) to confirm you're set.

You should now be equipped with the command to compile the code, and the command to run it!

Enter that in the cell below:

In [20]:
compile_command = ["clang++", "-std=c++17", "-Ofast", "-mcpu=native", "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]

## And now, on with the main task

In [21]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [22]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [23]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [24]:
def port(client, model, python):
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [25]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [26]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [14]:
run_python(pi)

Result: 3.141592656089
Execution Time: 45.561339 seconds


In [34]:
port(openai, OPENAI_MODEL, pi)

## CODE
```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

int main() {
    const long long iterations = 200000000LL;
    const double param1 = 4.0;
    const double param2 = 1.0;

    double result = 1.0;
    double i_times_p1 = param1; // equals 1*param1 initially

    auto start_time = std::chrono::high_resolution_clock::now();

    for (long long i = 0; i < iterations; ++i) {
        double denom1 = i_times_p1 - param2;
        result -= 1.0 / denom1;

        double denom2 = i_times_p1 + param2;
        result += 1.0 / denom2;

        i_times_p1 += param1;
    }

    double final_result = result * 4.0;

    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end_time - start_time;

    std::cout.setf(std::ios::fixed);
    std::cout << "Result: " << std::setprecision(12) << final_result << "\n";
    std::cout << "Execution Time: " << std::setprecision(6) << elapsed.count() << " seconds" << "\n";

    return 0;
}
```
## execution
Result: 3.141592656089

Execution Time: 1.531273 seconds


## **=== Code Execution Successful ===**

# Compiling C++ and executing

This next cell contains the command to compile a C++ file based on the instructions from GPT.

Again, it's not crucial to do this step if you don't wish to!

OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.  
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [16]:
# Use the commands from GPT 5

def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [ ]:
compile_and_run()

In [ ]:
19.178207/0.082168

## OK let's try the other contenders!

In [ ]:
port(anthropic, CLAUDE_MODEL, pi)
compile_and_run()

In [ ]:
port(grok, GROK_MODEL, pi)
compile_and_run()

In [26]:
def write_output2(cpp):
    with open("maing.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [27]:
def port2(client, model, python):
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output2(reply)

In [28]:
port2(gemini, GEMINI_MODEL, pi)

#code
```cpp

#include <iostream>
#include <iomanip>
#include <chrono>

// Function to perform the calculation
double calculate(int iterations, double param1, double param2) {
    double result = 1.0;
    // Using a for loop for iterations
    for (int i = 1; i <= iterations; ++i) {
        // Calculate j for subtraction
        double j_sub = static_cast<double>(i) * param1 - param2;
        // Calculate j for addition
        double j_add = static_cast<double>(i) * param1 + param2;
        
        // Avoid division by zero, although with the given parameters it's unlikely
        if (j_sub != 0) {
            result -= (1.0 / j_sub);
        }
        if (j_add != 0) {
            result += (1.0 / j_add);
        }
    }
    return result;
}

int main() {
    // Record start time
    auto start_time = std::chrono::high_resolution_clock::now();

    // Define parameters
    int iterations = 200000000;
    double param1 = 4.0;
    double param2 = 1.0;

    // Perform calculation and multiply by 4
    double final_result = calculate(iterations, param1, param2) * 4.0;

    // Record end time
    auto end_time = std::chrono::high_resolution_clock::now();
    // Calculate duration
    std::chrono::duration<double> elapsed_time = end_time - start_time;

    // Print the result with specified precision
    std::cout << std::fixed << std::setprecision(12) << "Result: " << final_result << std::endl;
    // Print the execution time with specified precision
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << elapsed_time.count() << " seconds" << std::endl;

    return 0;
}
```
## Execution
Result: 3.141592656089


Execution Time: 2.186025 seconds


## **=== Code Execution Successful ===**

In [ ]:
print(f"""
In Ed's experiments, the performance speedups were:

4th place: Claude Sonnet 4.5: {19.178207/0.104241:.0f}X speedup
3rd place: GPT-5: {19.178207/0.082168:.0f}X speedup
2nd place: Grok 4: {19.178207/0.018092:.0f}X speedup
1st place: Gemini 2.5 Pro: {19.178207/0.013314:.0f}X speedup
""")

# using local llm

In [27]:
import requests
requests.get("http://localhost:11434").content

OLLAMA_BASE_URL = "http://localhost:11434/v1"
from openai import OpenAI
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
MODEL = "llama3.2"

In [28]:
def write_output3(cpp):
    with open("main_ollama.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [29]:
def port3(client, model, python):
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output3(reply)

In [30]:
port3(ollama, MODEL, pi)

## llama3.2 code
```cpp
#include <iostream>
#include <unistd.h>

const long double param1 = 4.0;
const long double param2 = 1.0;

long double calculate(long double iterations) {
    long double result = 1.0;
    for (int i = 1; i <= static_cast<int>(iterations); ++i) {
        long double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result * param1;
}

int main() {
    // Avoid precision issues by using long double arithmetic
    std::cout.precision(18);

    auto start_time = std::chrono::high_resolution_clock::now();

    long int iterations = 200000000;
    auto result = calculate(iterations);

    auto end_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count() / 1000.0;

    // Print and display result in a human-readable format
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << duration / 60.0 << ".00 seconds" << std::endl;

    return 0;
}
```
# shows error
ERROR!
/tmp/M5427B7IXL/main.cpp: In function 'int main()':
/tmp/M5427B7IXL/main.cpp:23:28: error: 'std::chrono' has not been declared
   23 |     auto start_time = std::chrono::high_resolution_clock::now();
      |                            ^~~~~~
ERROR!
/tmp/M5427B7IXL/main.cpp:28:26: error: 'std::chrono' has not been declared
   28 |     auto end_time = std::chrono::high_resolution_clock::now();
      |                          ^~~~~~
/tmp/M5427B7IXL/main.cpp:29:26: error: 'std::chrono' has not been declared
   29 |     auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count() / 1000.0;
      |                          ^~~~~~
/tmp/M5427B7IXL/main.cpp:29:53: error: 'std::chrono' has not been declared
   29 |     auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count() / 1000.0;
      |                                                     ^~~~~~


## === Code Exited With Errors ===


In [36]:
#Using qwen2.5
def write_output4(cpp):
    with open("main_qwen2.5.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)


def port4(client, model, python):
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output4(reply)

In [37]:
port4(ollama, "qwen2.5", pi)

code qwen2.5
```cpp
#include <iostream>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        int j = i * param1 - param2;
        result -= 1.0 / j;
        j = i * param1 + param2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(200_000_000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << "Result: " << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " 
              << std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count() / 1'000'000.0
              << " seconds" << std::endl;

    return 0;
}
```
/tmp/nGs2LTLMjw/main.cpp: In function 'int main()':
/tmp/nGs2LTLMjw/main.cpp:17:31: error: unable to find numeric literal operator 'operator""_000_000'
   17 |     double result = calculate(200_000_000, 4, 1) * 4;
      |                               ^~~~~~~~~~~
/tmp/nGs2LTLMjw/main.cpp:20:37: error: 'setprecision' is not a member of 'std'
   20 |     std::cout << "Result: " << std::setprecision(12) << result << std::endl;
      |                                     ^~~~~~~~~~~~
/tmp/nGs2LTLMjw/main.cpp:3:1: note: 'std::setprecision' is defined in header '<iomanip>'; this is probably fixable by adding '#include <iomanip>'
    2 | #include <chrono>
  +++ |+#include <iomanip>
    3 | 


=== Code Exited With Errors ===

# local model
#llama error
#starcoder2 failed no code some sign of c++